Initialise code for google colab

Mount google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_addons as tfa
print(tfa.__version__)

2.11.0
0.19.0


In [1]:
!pip install tensorflow==2.11
!pip install tensorflow-addons==0.19

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorboard
    Found existing i

Create data base files under google colab environment

In [4]:
!unzip -q '/content/drive/MyDrive/data_equalize.zip' -d '/content/'

Define working directory to our jupyter repertory:
* because path to the different repertories (./data, ./output...) are define relatevly to jupyter one
* let import _mypath which add ./lib to python path in order to import our own define libraries


In [6]:
# for google colab use
%cd /content/drive/MyDrive/DS_MLOPS/jupyter
from google.colab.patches import cv2_imshow
db_work_dir = '/content'
work_dir = '..'

/content/drive/MyDrive/covid-19-xRay/jupyter


In [7]:
# for local use
db_work_dir = '../input/db/'
work_dir = '..'

In [2]:
import _mypath
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from PIL import Image
import joblib

%load_ext autoreload
%autoreload 1

2023-05-21 17:56:39.775506: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%aimport database.path_origin_data
%aimport database.dataset

from database.path_origin_data import build_data_paths 
from database.path_origin_data import lung_name, infection_name
from database.path_origin_data import train_name, test_name, valid_name
from database.path_origin_data import normal_name, covid_name, no_covid_name
from database.path_origin_data import images_name, lung_mask_name, infection_mask_name

from database.dataset import build_dataset

%aimport run_exp.test
%aimport run_exp.standard

from run_exp.test import compile_test_model, test_accuracy, test_conf_mat
from run_exp.standard import run_experiment as run_experiment_pure_cnn


/Users/luc/.local/share/virtualenvs/DS_MLOps-0EOZ04fs/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Build paths and variables for reading data base hierarchy

In [8]:
# db
db_name = 'data_add'
db_path = os.path.join(db_work_dir, db_name)

# input
input_path = os.path.join(work_dir, 'input')

data_name = 'data'
data_path = os.path.join(input_path, data_name)

model_name = 'model'
model_path = os.path.join(input_path, model_name)

# output
output_path = os.path.join('..', 'output', 'learning')
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)

log_path = os.path.join(output_path, 'log')
if not os.path.exists(log_path):
    os.makedirs(log_path, exist_ok=True)

ckpt_path = os.path.join(output_path, 'ckpt')
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path, exist_ok=True)

metric_path = os.path.join(output_path, 'metric')
if not os.path.exists(metric_path):
    os.makedirs(metric_path, exist_ok=True)

grad_cam_path = os.path.join(output_path, 'grad_cam')
if not os.path.exists(grad_cam_path):
    os.makedirs(grad_cam_path, exist_ok=True)


Structure to manage paths in data base

In [9]:
data_paths = build_data_paths()
idx = pd.IndexSlice

Create tf Dataset

In [10]:

paths = data_paths['path']

ds_train, ds_train_file_paths = build_dataset(db_path, paths, db=[lung_name], ds=[train_name])
ds_test, ds_test_file_paths = build_dataset(db_path, paths, db=[lung_name], ds=[test_name])
ds_valid, ds_valid_file_paths = build_dataset(db_path, paths, db=[lung_name], ds=[valid_name])
print(ds_train.cardinality().numpy())
print(len(ds_train_file_paths))


Found 1324 files belonging to 1 classes.
Found 1438 files belonging to 1 classes.
Found 1452 files belonging to 1 classes.
Found 400 files belonging to 1 classes.
Found 445 files belonging to 1 classes.
Found 450 files belonging to 1 classes.
Found 321 files belonging to 1 classes.
Found 364 files belonging to 1 classes.
Found 363 files belonging to 1 classes.
4214
4214


Model parameters

In [11]:
batch_size_cnn = 32
batch_size_transformer = 32

image_size = 256
input_shape = (image_size, image_size, 1)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100

label_smoothing = 0.1
lam_recon = 10.
patience = 5
min_delta = 0.005
min_delta_fine_tuning = 0.0005

# data augmentation
scale = 1. / 255.
flip = "horizontal"
rotation_factor = 10. / 360.
zoom_height_factor = 0.2
zoom_width_factor = 0.2

# vit
patch_size = 1
transformer_layers = 4
num_heads = 8
projection_dim = 64
transformer_units_rate = [2, 1]
mlp_head_units = [1024, 256]  # Size of the dense layers of the final classifier


Data augmentation

In [12]:
augmentation = keras.Sequential(
    [
      layers.Rescaling(scale=scale),
      layers.RandomFlip(flip),
      layers.RandomRotation(rotation_factor),
      layers.RandomZoom(height_factor=zoom_height_factor, width_factor=zoom_width_factor),
    ],
    name='augmentation'
)

cnn encoder

In [13]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout

encoder = keras.Sequential(
    [
      layers.Conv2D(128, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(128, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Flatten(),
    ],
    name='encoder'
)

In [14]:
classifier = keras.Sequential(
    [
      layers.Dense(1024, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(3, activation='softmax'),
    ],
    name='classifier'
)


In [15]:
# Sub Model
shared_encoder = keras.Sequential(name='shared_encoder')

for layer in encoder.layers[:-1]:
  shared_encoder.add(layer)

# for layer in shared_encoder.layers:
#   layer.trainable = False


In [16]:

%aimport myLayers.vision_transformer
%aimport myLayers.mlp
from myLayers.vision_transformer import add_vit

inputs = layers.Input(shape=input_shape)
augmented_transformer = augmentation(inputs)
shared_encoded = shared_encoder(augmented_transformer)
features = add_vit(shared_encoded,
            patch_size=patch_size,
            input_image_size=shared_encoded.shape[1],
            transformer_layers=transformer_layers,
            num_heads=num_heads,
            projection_dim=projection_dim,
            transformer_units_rate=transformer_units_rate,
            mlp_head_units=mlp_head_units)
# Classify outputs.
softmax = layers.Dense(3, activation='softmax', kernel_initializer='random_normal')(features)

# Create the Keras model
model = keras.Model(inputs=inputs, outputs=softmax)
# shared_encoder.summary()
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 augmentation (Sequential)      (None, 256, 256, 1)  0           ['input_1[0][0]']                
                                                                                                  
 shared_encoder (Sequential)    (None, 8, 8, 64)     296512      ['augmentation[0][0]']           
                                                                                                  
 patches (Patches)              (None, None, 64)     0           ['shared_encoder[0][0]']     

In [17]:
model_name = 'mlops_cnn_vit_model_weights.hdf5'
model_full_path = os.path.join(model_path, model_name)
model.load_weights(model_full_path)

In [18]:
%aimport run_exp.standard
from run_exp.standard import run_experiment as run_experiment_transformer

model_name = 'new_model'
transformer_history = run_experiment_transformer(
    model,
    ds_train, ds_valid, ds_test,
    batch_size=batch_size_transformer, num_epochs=num_epochs,
    learning_rate=learning_rate / 2., weight_decay=weight_decay,
    from_logits=False, label_smoothing=label_smoothing,
    patience=patience, min_delta=min_delta,
    log_path=log_path, ckpt_path=ckpt_path,
    prefix=model_name
)


Epoch 1/100


2023-05-21 17:59:06.416762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1324]
	 [[{{node Placeholder/_0}}]]
2023-05-21 17:59:06.417963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_25' with dtype string and shape [1452]
	 [[{{node Placeholder/_25}}]]


 16/132 [==>...........................] - ETA: 15:53 - loss: 0.6237 - accuracy: 0.8320

KeyboardInterrupt: 

In [ ]:
# model_name = 'new_model'
checkpoint_filename = os.path.join(ckpt_path, model_name + '_weights.hdf5')
model.load_weights(checkpoint_filename)


In [ ]:
y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model,
    ds_test, batch_size_transformer,
    from_logits=False, label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


213/213 [==============================] - 26s 72ms/step - loss: 0.4304 - accuracy: 0.9262
Test accuracy: 92.62%
213/213 [==============================] - 14s 59ms/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Predicted,0,1,2
Real,,,
0,2030,13,97
1,69,2234,91
2,213,18,2022
